In [1]:
import gensim

embeddings = gensim.models.KeyedVectors.load_word2vec_format("../GoogleNews-vectors-negative300.bin.gz", binary=True)

In [2]:
import pandas as pd

df = pd.read_csv("reviews_preprocessed.csv")
df.head()

,reviewText,asin,reviewerName,unixReviewTime,reviewerID,summary,score,helpfulP,helpfulN
0,"I enjoyed the ""Lite"" game, but it's not (as ad...",3243,13241,1329177600,5183,Too bad :(.,1.0,1,2
1,"Didn't like it hard to aim and shot, not going...",9415,21314,1400803200,50110,Bubble Blaze,1.0,0,0
2,Nothing great about this. I think I deleted it...,1316,43046,1381795200,46108,Just another app,1.0,1,1
3,this is a really bad program with poor graphic...,378,26092,1309737600,38875,Junk,1.0,0,0
4,"I really like the game, but when I got to leve...",2396,39069,1381363200,58013,fun till level 5,1.0,1,1


In [3]:
import re
import numpy as np
import nltk

stopwords = nltk.corpus.stopwords.words('english')


def get_tokens(text: str):
    tokens = nltk.word_tokenize(text)
    stems = []

    # porter = nltk.PorterStemmer()
    for token in tokens:
        if re.search("\W", token) is not None or token in stopwords:
            continue
        # stems.append(porter.stem(token))
        stems.append(token)
    return stems


def get_doc_vectors(df_data: pd.DataFrame):
    vectors_arr = np.ndarray((df_data.shape[0], embeddings.vector_size), dtype=np.float32)
    for i in df_data.index:
        tokens = get_tokens(df_data[i])
        temp = np.ndarray((len(tokens), embeddings.vector_size), dtype=np.float32)
        for j in range(len(tokens)):
            if tokens[j] in embeddings:
                temp[j] = embeddings[tokens[j]]

        temp_mean = temp.mean(axis=0)
        temp_mean = np.nan_to_num(temp_mean)
        vectors_arr[i] = temp_mean

    return vectors_arr


reviewText_vectors = get_doc_vectors(df['reviewText'])
summary_vectors = get_doc_vectors(df['summary'])
df_docs_vectors = pd.DataFrame(np.concatenate([reviewText_vectors, summary_vectors], axis=1))
df_docs_vectors.head()

c:\users\festo\onedrive - put.poznan.pl\semestr ix\emd\emd-python\venv\lib\site-packages\numpy\core\_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\festo\AppData\Local\Temp\ipykernel_8264\3351094019.py:30: RuntimeWarning: Mean of empty slice.
  temp_mean = temp.mean(axis=0)
c:\users\festo\onedrive - put.poznan.pl\semestr ix\emd\emd-python\venv\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599
0,0.070107,0.009079,-0.039810,0.058842,-0.043427,0.033958,6.595421e-02,-4.783249e-02,0.072006,0.076176,...,0.043762,0.277344,-0.165771,0.078613,-0.066284,-0.068848,-0.073029,-0.064209,0.074341,0.171753
1,0.102240,0.120047,0.076555,0.112930,-0.000720,0.022008,1.065877e-01,-9.454855e-02,0.073934,0.080598,...,0.000000,-0.097778,0.051758,-0.172974,-0.000977,-0.160767,-0.106689,0.074463,-0.235840,-0.159912
2,-0.036605,0.018754,0.065934,0.153353,-0.004134,-0.024316,5.400391e-02,-6.633708e-02,0.112891,0.094499,...,0.059448,0.109619,-0.127553,-0.042399,0.046402,-0.051860,0.015299,-0.126709,-0.091634,-0.004720
3,0.048904,351224.468750,0.031008,0.105335,-0.022068,0.054226,1.885335e+25,4.663174e+34,0.043652,0.049229,...,-0.001389,0.155273,0.046631,0.184570,0.107910,-0.007629,-0.244141,-0.206055,-0.060303,-0.151367
4,0.035550,0.073834,0.036061,0.106156,0.019434,0.018611,7.603737e-02,-1.133717e-01,0.049721,0.101280,...,-0.098114,0.169250,-0.137939,-0.012421,-0.096619,0.060303,-0.071228,-0.059097,-0.104614,0.017197


In [4]:
# Sprawdzanie czy w danych nie ma nieleganych wartości
df_docs_vectors[df_docs_vectors.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599


In [5]:
df_result = pd.concat([df[list(set(df.columns) - {"reviewText", "summary"})], df_docs_vectors], axis=1)
df_result.columns = df_result.columns.astype(str)
df_result.to_csv("reviews_w2v.csv", index=False)